# Praktikum Deep Learning — Transfer Learning (Pengantar & Demo Konsep)

_Notebook ini memandu kamu memahami konsep inti transfer learning sebelum terjun ke studi kasus aktual. Gunakan sebagai pijakan awal: jalankan sel satu per satu, catat pengamatan, dan refleksikan apa yang terjadi._

> Tips: aktifkan menu **View → Table of Contents** di JupyterLab supaya navigasi antar bagian lebih mudah.

---

**Profil Modul**
- **Untuk siapa?** Mahasiswa yang sudah nyaman dengan CNN dasar dan Python.
- **Durasi saran:** 2 x 50 menit (teori interaktif 40 menit, demo terarah 35 menit, refleksi 25 menit).
- **Peran Notebook:** panduan belajar mandiri sekaligus skrip demonstrasi yang dapat kamu ulangi setelah sesi.

**Mengapa Modul Ini Penting**
- Kamu akan melihat langsung bagaimana reuse model pretrained menghemat waktu eksperimen.
- Transfer learning membuatmu melangkah lebih jauh dengan data terbatas—ibarat sudah bisa berjalan, lalu belajar berlari tanpa harus tumbuh dari bayi lagi.
- Bayangkan pemain biola yang beralih ke pemrograman: ia tidak mengulang belajar ritme dari nol, melainkan memindahkan sensitivitas tempo ke logika kode. Transfer learning bekerja dengan cara serupa, membawa keterampilan lama ke domain baru.

**Apa yang Akan Kamu Lakukan**
1. Menyegarkan kembali motivasi transfer learning dan menautkannya dengan pengalaman belajarmu.
2. Menjalankan eksperimen terstruktur: konfigurasi, dataset dummy, dan modifikasi backbone.
3. Menganalisis metrik dan mengaitkannya dengan strategi fine-tuning.
4. Menyusun ide perluasan ringan untuk eksplorasi pribadi.

**Prasyarat Teknis**
- Memahami supervised learning, metrik akurasi, dan konsep overfitting.
- Familiar dengan komponen PyTorch dasar (`DataLoader`, `nn.Module`, training loop).
- Sudah memasang dependensi pada `requirements.txt`.

**Hasil Belajar yang Diharapkan**
- Catatan refleksi kapan memilih freeze vs fine-tuning.
- Screenshot atau langkah analisis plot loss-akurasi di setiap tahap.
- Draft ide adaptasi model untuk dataset pilihanmu sendiri.


## Navigasi & Cara Menggunakan Notebook

Ikuti alur berikut agar pengalaman latihan terasa ringan namun efektif:
1. **Baca cepat bagian A–C.** Catat tujuan utama dan ringkasan teori supaya kamu tahu apa yang sedang dicari.
2. **Jalankan blok kode secara berurutan mulai dari D.** Setiap blok sudah diberi log sehingga kamu tahu status terbaru.
3. **Perhatikan output di bagian F dan G.** Di sana kamu akan melihat jumlah sampel subset serta banyaknya parameter yang benar-benar dilatih.
4. **Amati log training di bagian H.** Log `[feature_extraction]` dan `[fine_tuning]` menunjukkan progress epoch tanpa perlu buka plot terlebih dulu.
5. **Gunakan bagian I–N untuk refleksi.** Plot, checklist, dan glosarium membantu mengikat hasil eksperimen ke konsep teori.

> Tips cepat: jika waktu sangat terbatas, jalankan hanya hingga plot pada bagian I; sisanya bisa dibaca untuk pemahaman tanpa eksekusi ulang.


## A. Judul & Tujuan Pembelajaran

Gunakan bagian ini untuk menyelaraskan harapan. Bacalah tujuan berikut, lalu tuliskan (misalnya di jurnal atau catatan digital) dua pertanyaan yang ingin kamu jawab setelah praktikum selesai.

**Learning Objectives**
- Memahami konsep dan motivasi Transfer Learning (hemat data, waktu, serta sumber daya komputasi) dan posisinya dalam pipeline deep learning modern.
- Menjelaskan kelebihan utama pendekatan ini: efisiensi training, kebutuhan data lebih sedikit, performa stabil, dan mitigasi overfitting.
- Mengidentifikasi variasi Transfer Learning (Inductive, Transductive, Unsupervised) serta kapan masing-masing cocok dipakai.
- Membedakan mode feature extraction (freeze) vs fine-tuning penuh atau parsial, termasuk pengaruhnya pada jumlah parameter yang di-update.
- Mengaplikasikan konfigurasi PyTorch untuk memuat backbone pretrained dan menyesuaikan classifier head.

**Setelah Selesai Kamu Seharusnya Bisa**
- Menyebutkan minimal dua alasan transfer learning relevan untuk dataset aktual.
- Menganalisis kapan harus membuka layer tambahan ketika domain target jauh dari domain sumber.
- Memodifikasi hyperparameter dasar (learning rate, `freeze_until`) dan menuliskan implikasinya.

**Pemanasan Cepat**
- Bayangkan proses belajar berjalan: kamu tidak mengulang dari merangkak setiap kali belajar olahraga baru. Transfer learning bekerja dengan prinsip serupa. Catat analogi versimu sendiri di catatan praktikum.


## B. Recall & Icebreaker

Luangkan 10 menit untuk kembali ke pengalamanmu sendiri. Jika belajar berkelompok, diskusikan jawaban; jika mandiri, tuliskan poin-poin penting.

1. Apa tantangan melatih CNN dari nol ketika dataset kecil? Hubungkan dengan pengalamanmu memulai dari hal yang benar-benar baru tanpa referensi (ibarat belajar berjalan tanpa contoh).
2. Mengapa reuse pengetahuan dari model besar masuk akal? Bayangkan kamu sudah mahir berjalan; belajar berlari atau berdansa jadi lebih mudah karena fondasinya sama.
3. Kapan cukup melakukan freeze? Kapan perlu fine-tune beberapa layer terakhir? Refleksikan situasi ketika kamu harus meningkatkan kemampuan dari berjalan santai ke sprint karena tuntutan tugas berbeda.


## C. Ringkasan Teori

Transfer learning adalah pendekatan reuse model terlatih sebagai titik awal agar hemat komputasi, data, dan waktu; kamu tidak selalu harus memulai training dari nol. Intinya, representasi fitur yang sudah kaya dipakai ulang lalu disesuaikan pada tugas baru.

**Manfaat & Kelebihan**
- **Efisiensi training:** mengurangi epochs dan percobaan trial-and-error.
- **Kebutuhan data lebih sedikit:** cocok untuk domain dengan data mahal seperti medis.
- **Stabilitas performa:** membantu model konvergen pada solusi yang lebih baik.
- **Mitigasi overfitting:** pengetahuan awal dari dataset besar bertindak sebagai regularizer.

**Jenis-jenis Transfer Learning**
1. *Inductive TL:* label tersedia di domain target; tugas sumber boleh berbeda (contoh: pretraining ImageNet lalu klasifikasi retinal fundus).
2. *Transductive TL:* label target minim, domain sumber mirip; fokus pada adaptasi distribusi (contoh: domain adaptation).
3. *Unsupervised TL:* reuse representasi tanpa label target, lalu gunakan metode lain di atas embedding.

**Strategi Adaptasi Model**
- **Feature Extraction (Freeze):** membekukan backbone, melatih classifier head baru. Analoginya: kamu sudah mahir berjalan, cukup belajar memegang benda sambil berjalan.
- **Fine-Tuning Parsial:** membuka beberapa blok terakhir untuk menyesuaikan fitur tingkat tinggi—seperti belajar jogging ringan setelah lama berjalan agar tubuh menyesuaikan beban baru.
- **Fine-Tuning Penuh:** melatih seluruh backbone; ibarat melatih ulang teknik berlari cepat dari nol karena kamu berpindah ke olahraga yang berbeda jauh.

**Istilah Penting**
- *Backbone:* model dasar penyedia fitur.
- *Classifier Head:* lapisan akhir sesuai tugas target.
- *Layer Freezing:* menonaktifkan grad agar bobot tetap.
- *Domain Gap:* beda karakteristik antara data sumber dan target; makin besar gap, makin banyak adaptasi diperlukan.

Catat analogi versi kamu sendiri untuk tiap strategi agar konsepnya melekat.


## D. Setup Lingkungan

Bagian ini memastikan lingkungan eksekusi siap sebelum kamu menjalankan eksperimen.

**Langkah Kamu**
1. Jalankan kode di bawah untuk memuat dependensi dan utility.
2. Perhatikan pesan versi Python, PyTorch, dan ketersediaan GPU.
3. Pastikan folder `outputs/` dibuat otomatis; gunakan untuk menyimpan artefak.

**Checklist Pribadi**
- Sudah melakukan `pip install -r requirements.txt`.
- Environment (conda/virtualenv) aktif denga versi paket sesuai kebutuhan.
- Jika hanya punya CPU, berdoa saja agar bisa beli GPU.


In [20]:
import json
import os
import platform
import random
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import datasets, models, transforms
from torchvision.transforms import functional as F_transforms
import yaml

# Fungsi utilitas agar hasil eksperimen deterministik untuk kebutuhan praktikum.
def set_seed(seed: int = 42) -> None:
    """Set random seeds untuk numpy, random, dan torch agar hasil reproducible."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def get_device(preference: str = "cuda_if_available") -> torch.device:
    """Pilih device berdasarkan preferensi dan ketersediaan CUDA."""
    if preference == "cuda_if_available" and torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

# Deteksi root project (notebook berada di folder notebooks/).
project_root = Path.cwd().resolve()
if project_root.name == "notebooks":
    project_root = project_root.parent

paths_to_create = [
    project_root / "outputs" / "figures",
    project_root / "outputs" / "reports",
    project_root / "models",
]
for path in paths_to_create:
    path.mkdir(parents=True, exist_ok=True)

# Simpan cache weight torchvision ke folder models/ agar rapih (dan reusable jika tersedia).
os.environ.setdefault("TORCH_HOME", str(project_root / "models"))

print(f"Project root: {project_root}")
print(f"Python version: {platform.python_version()}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

seed_value = 42
set_seed(seed_value)

device = get_device()
print(f"Seed set to: {seed_value}")
print(f"Selected device: {device}")


Project root: /home/juni/Praktikum/deep-learning/transfer-learning-practicum
Python version: 3.12.3
PyTorch version: 2.8.0+cu128
CUDA available: False
Seed set to: 42
Selected device: cpu


## E. Konfigurasi (Code)

Konfigurasi dipisahkan melalui file YAML agar eksperimen mudah direplikasi dan dipersonalisasi.

**Apa yang Harus Kamu Lakukan**
- Baca konfigurasi yang ditampilkan sebagai tabel untuk memahami parameter default (jumlah kelas, ukuran input, strategi freeze).
- Catat bagaimana `num_workers`, `batch_size`, dan `learning_rate` mempengaruhi waktu training.
- Ubah satu parameter (misalnya `learning_rate`) dan dokumentasikan dampaknya sebagai bagian refleksi.

Anggap konfigurasi ini seperti rencana latihan: kamu bisa menyesuaikan intensitas (learning rate) ketika merasa sudah siap “berlari” lebih cepat.


In [21]:
# Membaca konfigurasi template agar eksperimen mudah direplikasi.
config_path = project_root / "configs" / "training.yaml"
with config_path.open("r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

# Menormalkan nilai num_workers agar aman dijalankan lintas OS / notebook.
num_workers_cfg = int(config.get("num_workers", 0))
if os.name == "nt":
    num_workers_cfg = 0  # Windows + notebook lebih stabil single-thread loader.
if num_workers_cfg < 0:
    num_workers_cfg = 0
config["num_workers"] = num_workers_cfg

config_df = pd.DataFrame(list(config.items()), columns=["parameter", "value"]).set_index("parameter")
display(config_df)

seed_value = config.get("seed", seed_value)
set_seed(seed_value)
device = get_device(config.get("device", "cuda_if_available"))

data_root_path = (project_root / config.get("data_root", "data/raw")).resolve()
data_root_path.mkdir(parents=True, exist_ok=True)

print(f"Konfigurasi dimuat dari: {config_path}")
print(f"Seed aktif: {seed_value}")
print(f"Device aktif: {device}")
print(f"num_workers efektif: {config['num_workers']}")
print(f"Data root: {data_root_path}")


,value
parameter,
seed,42
device,cuda_if_available
batch_size,16
num_workers,2
num_epochs_feature_extraction,2
num_epochs_fine_tuning,2
optimizer,adam
lr_feature_extraction,1e-3
lr_fine_tuning,1e-4


Konfigurasi dimuat dari: /home/juni/Praktikum/deep-learning/transfer-learning-practicum/configs/training.yaml
Seed aktif: 42
Device aktif: cpu
num_workers efektif: 2
Data root: /home/juni/Praktikum/deep-learning/transfer-learning-practicum/data/raw


## F. DataModule Real-World (Code)

Pipeline data kini menggunakan dataset nyata untuk latihan. Secara default notebook memuat CIFAR-10 melalui `torchvision.datasets`,
n kemudian membatasi sebagian sampel agar latihan cepat dijalankan di kelas/lab. Kamu dapat menggantinya dengan dataset
berbasis `ImageFolder` (struktur `train/` dan `val/`) tanpa perlu mengubah logika training yang lain.

**Langkah Mahasiswa**
- Pastikan koneksi internet aktif saat pertama kali mengunduh CIFAR-10, atau siapkan salinan lokal di `data/raw`.
- Ubah parameter `train_subset_fraction` dan `val_subset_fraction` pada `configs/training.yaml` jika ingin menggunakan seluruh dataset.
- Jika memakai dataset sendiri, set `dataset_name: imagefolder` dan arahkan `data_root` ke direktori yang memuat subfolder `train/` dan `val/`.

Bayangkan proses ini seperti berlatih dari lagu nyata setelah menyiapkan teknik dasar: kita tidak lagi memainkan nada acak,
melainkan menyesuaikan keterampilan ke karya yang sesungguhnya.

**Ringkasan cepat:** notebook otomatis membatasi sampel (parameter `train_subset_fraction`, `max_train_samples`). Lihat log setelah menjalankan sel ini untuk memastikan jumlah batch tidak berlebihan.


In [22]:
# DataModule untuk dataset citra nyata (CIFAR-10 sebagai default, ImageFolder sebagai opsi lanjutan).
class ImageClassificationDataModule:
    """Memuat dataset klasifikasi citra dan menyediakan DataLoader untuk training/validasi."""

    def __init__(
        self,
        dataset_name: str,
        data_root: Path,
        batch_size: int,
        num_workers: int,
        seed: int,
        train_subset_fraction: float = 1.0,
        val_subset_fraction: float = 1.0,
        max_train_samples: int | None = None,
        max_val_samples: int | None = None,
        download: bool = True,
    ) -> None:
        self.dataset_name = dataset_name.lower()
        self.data_root = data_root
        self.batch_size = batch_size
        self.num_workers = max(0, int(num_workers))
        if os.name == "nt":
            self.num_workers = 0
        self.seed = seed
        self.train_subset_fraction = float(train_subset_fraction)
        self.val_subset_fraction = float(val_subset_fraction)
        self.max_train_samples = None if max_train_samples is None else int(max_train_samples)
        self.max_val_samples = None if max_val_samples is None else int(max_val_samples)
        self.download = bool(download)

        self.train_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        self.eval_transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        self.train_dataset = None
        self.val_dataset = None
        self.num_classes = None

    def setup(self) -> None:
        if self.dataset_name == "cifar10":
            train_full = datasets.CIFAR10(
                root=self.data_root,
                train=True,
                transform=self.train_transforms,
                download=self.download,
            )
            val_full = datasets.CIFAR10(
                root=self.data_root,
                train=False,
                transform=self.eval_transforms,
                download=self.download,
            )
            self.num_classes = len(train_full.classes)
        elif self.dataset_name == "imagefolder":
            train_dir = self.data_root / "train"
            val_dir = self.data_root / "val"
            if not train_dir.exists() or not val_dir.exists():
                raise FileNotFoundError(
                    "Direktori imagefolder tidak ditemukan. Pastikan memiliki subfolder 'train/' dan 'val/'."
                )
            train_full = datasets.ImageFolder(train_dir, transform=self.train_transforms)
            val_full = datasets.ImageFolder(val_dir, transform=self.eval_transforms)
            self.num_classes = len(train_full.classes)
        else:
            raise ValueError("dataset_name hanya mendukung 'cifar10' atau 'imagefolder' untuk saat ini.")

        self.train_dataset = self._maybe_subset(
            train_full,
            fraction=self.train_subset_fraction,
            max_samples=self.max_train_samples,
        )
        self.val_dataset = self._maybe_subset(
            val_full,
            fraction=self.val_subset_fraction,
            max_samples=self.max_val_samples,
        )

    def _maybe_subset(self, dataset, fraction: float, max_samples: int | None):
        total_len = len(dataset)
        target_len = total_len
        fraction = float(max(0.0, min(fraction, 1.0)))
        if fraction < 1.0:
            target_len = max(1, int(total_len * fraction))
        if max_samples is not None:
            target_len = max(1, min(target_len, max_samples))
        if target_len >= total_len:
            return dataset
        generator = torch.Generator().manual_seed(self.seed)
        indices = torch.randperm(total_len, generator=generator)[:target_len].tolist()
        return Subset(dataset, indices)

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=torch.cuda.is_available(),
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=torch.cuda.is_available(),
        )


dataset_name_cfg = config.get("dataset_name", "cifar10")
train_subset_fraction = config.get("train_subset_fraction", 1.0)
val_subset_fraction = config.get("val_subset_fraction", 1.0)
max_train_samples = config.get("max_train_samples")
max_val_samples = config.get("max_val_samples")
download_dataset = config.get("download_dataset", True)

datamodule = ImageClassificationDataModule(
    dataset_name=dataset_name_cfg,
    data_root=data_root_path,
    batch_size=config["batch_size"],
    num_workers=config["num_workers"],
    seed=seed_value,
    train_subset_fraction=train_subset_fraction,
    val_subset_fraction=val_subset_fraction,
    max_train_samples=max_train_samples,
    max_val_samples=max_val_samples,
    download=download_dataset,
)

datamodule.setup()
NUM_CLASSES = datamodule.num_classes

sample_train_batch = next(iter(datamodule.train_dataloader()))
sample_val_batch = next(iter(datamodule.val_dataloader()))
print(f"Dataset: {dataset_name_cfg.upper()} | kelas: {NUM_CLASSES}")
print(
    f"Train subset: {len(datamodule.train_dataset)} sampel (batch {sample_train_batch[0].shape})"
)
print(
    f"Val subset: {len(datamodule.val_dataset)} sampel (batch {sample_val_batch[0].shape})"
)


Dataset: CIFAR10 | kelas: 10
Train subset: 1024 sampel (batch torch.Size([16, 3, 224, 224]))
Val subset: 512 sampel (batch torch.Size([16, 3, 224, 224]))


## G. Bangun Model Pretrained (Code)

Mengambil backbone pretrained (`resnet18`), memisahkan feature extractor vs classifier, serta menyiapkan fungsi freeze atau unfreeze untuk mode feature extraction dan fine-tuning.

**Hal yang Perlu Diamati**
- Perhatikan bagaimana fungsi `build_backbone` mengganti classifier head.
- Pikirkan arti parameter `pretrained=True` dan sumber bobot (ImageNet).
- Modifikasi `NUM_CLASSES` lalu amati perubahan ukuran layer akhir.

Analoginya, backbone pretrained adalah otot dasar yang sudah terlatih berjalan. Kamu menambahkan “skill baru” (classifier head) agar bisa berlari mengikuti lomba tertentu.


In [23]:
# Utilitas pemodelan untuk memisahkan backbone dan classifier.
def build_backbone(name: str = "resnet18", pretrained: bool = True, num_classes: int | None = None):
    """Membangun backbone torchvision dan memisahkan classifier head."""
    if name != "resnet18":
        raise ValueError("Demo ini saat ini hanya mendukung resnet18 sebagai baseline ringan.")

    if num_classes is None:
        if "NUM_CLASSES" not in globals():
            raise ValueError("NUM_CLASSES belum terdefinisi. Jalankan sel DataModule terlebih dahulu.")
        num_classes = NUM_CLASSES

    base_model = None
    weights_info = "random-init"
    if pretrained:
        try:
            if hasattr(models, "ResNet18_Weights"):
                base_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
                weights_info = "ResNet18_Weights.DEFAULT"
            else:
                base_model = models.resnet18(pretrained=True)
                weights_info = "pretrained=True"
        except Exception as exc:
            print(f"Gagal memuat weight pretrained (offline?): {exc}")
            weights_info = "random-init (fallback)"

    if base_model is None:
        if hasattr(models, "ResNet18_Weights"):
            base_model = models.resnet18(weights=None)
        else:
            base_model = models.resnet18(pretrained=False)

    feature_extractor = nn.Sequential(*list(base_model.children())[:-1])
    in_features = base_model.fc.in_features
    classifier = nn.Linear(in_features, num_classes)
    return feature_extractor, classifier, weights_info


class TransferLearner(nn.Module):
    """Model wrapper yang memisahkan feature extractor dan classifier."""
    def __init__(self, feature_extractor: nn.Module, classifier: nn.Module) -> None:
        super().__init__()
        self.feature_extractor = feature_extractor
        self.classifier = classifier

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = self.feature_extractor(x)
        features = torch.flatten(features, 1)
        return self.classifier(features)


def set_feature_extractor_grad(feature_extractor: nn.Module, freeze_until: str = "all") -> None:
    """Atur parameter backbone yang dapat di-train sesuai kebijakan freeze."""
    freeze_until = freeze_until.lower()
    if freeze_until not in {"all", "layer4", "none"}:
        raise ValueError("freeze_until harus salah satu dari: all | layer4 | none")

    for param in feature_extractor.parameters():
        param.requires_grad = False

    if freeze_until == "layer4":
        for name, module in feature_extractor.named_children():
            if name == "7":
                for param in module.parameters():
                    param.requires_grad = True
    elif freeze_until == "none":
        for param in feature_extractor.parameters():
            param.requires_grad = True


def count_trainable_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


feature_extractor, classifier, weights_info = build_backbone(
    name=config.get("pretrained_backbone", "resnet18"),
    pretrained=True,
    num_classes=NUM_CLASSES,
)
print(f"Backbone weight source: {weights_info}")
model = TransferLearner(feature_extractor, classifier).to(device)

# Mode feature extraction: seluruh backbone di-freeze.
set_feature_extractor_grad(model.feature_extractor, freeze_until="all")
fe_params = count_trainable_parameters(model)
print(f"Trainable params (feature extraction): {fe_params}")

# Mode fine-tuning: buka sesuai konfigurasi freeze_until.
set_feature_extractor_grad(model.feature_extractor, freeze_until=config.get("freeze_until"))
ft_params = count_trainable_parameters(model)
print(f"Trainable params (fine-tuning policy '{config.get('freeze_until', 'all')}'): {ft_params}")


Backbone weight source: ResNet18_Weights.DEFAULT
Trainable params (feature extraction): 5130
Trainable params (fine-tuning policy 'layer4'): 8398858


## H. Loop Train Generic (Code)

Loop training ini mendemokan dua tahap: feature extraction (melatih classifier saja) dan fine-tuning (opsional membuka sebagian backbone). Logging masih sederhana karena dataset dummy.

**Panduan Praktik**
- Jalankan sel training dan amati output loss/akurasi setiap epoch.
- Tuliskan perbedaan dinamika antara tahap feature extraction dan fine-tuning.
- Eksperimen: ubah jumlah epoch atau buka lebih banyak layer lalu bandingkan hasilnya.

Pikirkan transisi freeze → fine-tune seperti beralih dari jalan santai ke lari interval: kamu perlu menyesuaikan kecepatan, napas (learning rate), dan durasi agar tubuh (model) tetap stabil.

**Cara membaca output:** perhatikan baris `[feature_extraction]` dan `[fine_tuning]` yang muncul di terminal. Itu menunjukkan epoch, loss, dan akurasi sehingga kamu bisa menghentikan eksekusi bila sudah cukup.


In [24]:
# Fungsi training sederhana agar pipeline dapat dijalankan end-to-end.
def train_one_epoch(model: nn.Module, dataloader: DataLoader, criterion, optimizer, device: torch.device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / max(total, 1)
    epoch_acc = correct / max(total, 1)
    return epoch_loss, epoch_acc


def evaluate(model: nn.Module, dataloader: DataLoader, criterion, device: torch.device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    epoch_loss = running_loss / max(total, 1)
    epoch_acc = correct / max(total, 1)
    return epoch_loss, epoch_acc


def run_training_cycles(model: nn.Module, datamodule: ImageClassificationDataModule, config: dict, device: torch.device):
    history = []
    criterion = nn.CrossEntropyLoss()
    train_loader = datamodule.train_dataloader()
    val_loader = datamodule.val_dataloader()

    lr_fe = float(config["lr_feature_extraction"])
    lr_ft = float(config["lr_fine_tuning"])
    weight_decay = float(config["weight_decay"])

    # Tahap 1: Feature Extraction (freeze backbone).
    feature_epochs = int(config["num_epochs_feature_extraction"])
    set_feature_extractor_grad(model.feature_extractor, freeze_until="all")
    optimizer_fe = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=lr_fe,
        weight_decay=weight_decay,
    )
    for epoch in range(1, feature_epochs + 1):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer_fe, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        history.append({
            "stage": "feature_extraction",
            "epoch": epoch,
            "train_loss": float(train_loss),
            "train_acc": float(train_acc),
            "val_loss": float(val_loss),
            "val_acc": float(val_acc),
        })
        print(
            f"[feature_extraction] Epoch {epoch}/{feature_epochs} — "
            f"train_loss={train_loss:.4f} val_loss={val_loss:.4f} "
            f"train_acc={train_acc:.3f} val_acc={val_acc:.3f}"
        )

    # Tahap 2: Fine-tuning (opsional membuka layer backbone).
    fine_tune_epochs = int(config.get("num_epochs_fine_tuning", 0))
    if fine_tune_epochs > 0:
        set_feature_extractor_grad(model.feature_extractor, freeze_until=config.get("freeze_until", "all"))
        optimizer_ft = optim.Adam(
            filter(lambda p: p.requires_grad, model.parameters()),
            lr=lr_ft,
            weight_decay=weight_decay,
        )
        for epoch in range(1, fine_tune_epochs + 1):
            train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer_ft, device)
            val_loss, val_acc = evaluate(model, val_loader, criterion, device)
            history.append({
                "stage": "fine_tuning",
                "epoch": epoch,
                "train_loss": float(train_loss),
                "train_acc": float(train_acc),
                "val_loss": float(val_loss),
                "val_acc": float(val_acc),
            })
            print(
                f"[fine_tuning] Epoch {epoch}/{fine_tune_epochs} — "
                f"train_loss={train_loss:.4f} val_loss={val_loss:.4f} "
                f"train_acc={train_acc:.3f} val_acc={val_acc:.3f}"
            )
    else:
        print("Tahap fine_tuning dilewati (num_epochs_fine_tuning=0).")

    return history


model = model.to(device)
history = run_training_cycles(model, datamodule, config, device)
print(f"Selesai training demo dengan {len(history)} entry riwayat.")


[feature_extraction] Epoch 1/2 — train_loss=2.1256 val_loss=1.7530 train_acc=0.249 val_acc=0.389
[feature_extraction] Epoch 2/2 — train_loss=1.4771 val_loss=1.3619 train_acc=0.527 val_acc=0.527
[fine_tuning] Epoch 1/2 — train_loss=0.9324 val_loss=0.7697 train_acc=0.702 val_acc=0.732
[fine_tuning] Epoch 2/2 — train_loss=0.4932 val_loss=0.6940 train_acc=0.842 val_acc=0.756
Selesai training demo dengan 4 entry riwayat.


## I. Plot & Logging (Code)

Visualisasi metrik dummy dan simpan artefak (plot serta ringkasan JSON) ke folder `outputs/` agar mudah diperiksa setelah praktikum.

**Langkah Observasi**
- Bandingkan kurva loss antar tahap; tuliskan observasi di jurnal belajar.
- Screenshot tabel `history_df` atau catat angka penting (misal stagnasi akurasi) sebagai bahan diskusi.
- Buka file JSON atau figur di `outputs/` lalu verifikasi penamaan file sesuai standar tim.

Gunakan analogi latihan fisik: grafik loss adalah detak jantungmu. Saat mulai “berlari” (fine-tune), apakah detak meningkat (loss naik) sebelum turun stabil? Catat interpretasimu.

**Sebelum lanjut:** pastikan file pada `outputs/figures` dan `outputs/reports` sudah terbuat. Jika belum, ulangi sel training lalu jalankan sel ini lagi.


In [25]:
# Menyusun history ke DataFrame untuk analisis cepat.
history_df = pd.DataFrame(history)
if not history_df.empty:
    display(history_df)

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    for stage in history_df["stage"].unique():
        subset = history_df[history_df["stage"] == stage]
        axes[0].plot(subset["epoch"], subset["train_loss"], marker="o", label=f"{stage} train")
        axes[0].plot(subset["epoch"], subset["val_loss"], marker="s", label=f"{stage} val")
        axes[1].plot(subset["epoch"], subset["train_acc"], marker="o", label=f"{stage} train")
        axes[1].plot(subset["epoch"], subset["val_acc"], marker="s", label=f"{stage} val")
    axes[0].set_title("Loss per Stage")
    axes[0].set_xlabel("Epoch")
    axes[0].set_ylabel("Loss")
    axes[0].legend()
    axes[1].set_title("Accuracy per Stage")
    axes[1].set_xlabel("Epoch")
    axes[1].set_ylabel("Accuracy")
    axes[1].legend()
    plt.tight_layout()

    fig_path = project_root / config["fig_dir"] / "loss_accuracy_demo.png"
    fig.savefig(fig_path, bbox_inches="tight")
    plt.close(fig)
    print(f"Plot tersimpan di: {fig_path}")
else:
    print("History kosong: tidak ada data untuk divisualisasikan.")

summary = {
    "device": str(device),
    "config": config,
    "history": history,
}
summary_path = project_root / config["log_dir"] / "run_summary.json"
with summary_path.open("w", encoding="utf-8") as f:
    json.dump(summary, f, indent=2)
print(f"Ringkasan run tersimpan di: {summary_path}")

model_path = project_root / config["save_dir"] / "demo_model_state_dict.pt"
torch.save(model.state_dict(), model_path)
print(f"Model checkpoint dummy tersimpan di: {model_path}")


,stage,epoch,train_loss,train_acc,val_loss,val_acc
0,feature_extraction,1,2.125630,0.249023,1.753035,0.388672
1,feature_extraction,2,1.477140,0.527344,1.361905,0.527344
2,fine_tuning,1,0.932377,0.702148,0.769742,0.732422
3,fine_tuning,2,0.493241,0.841797,0.694042,0.755859


Plot tersimpan di: /home/juni/Praktikum/deep-learning/transfer-learning-practicum/outputs/figures/loss_accuracy_demo.png
Ringkasan run tersimpan di: /home/juni/Praktikum/deep-learning/transfer-learning-practicum/outputs/reports/run_summary.json
Model checkpoint dummy tersimpan di: /home/juni/Praktikum/deep-learning/transfer-learning-practicum/models/demo_model_state_dict.pt


## J. Ringkasan & Diskusi

| Mode | Kecepatan | Kebutuhan Data | Risiko Overfitting | Kapan Dipilih |
| --- | --- | --- | --- | --- |
| Feature Extraction (Freeze) | Cepat (hanya train head) | Rendah | Rendah | Dataset kecil, domain mirip |
| Fine-Tuning Parsial | Sedang (beberapa layer dibuka) | Menengah | Menengah | Saat butuh adaptasi moderat, layer akhir di-unfreeze |
| Fine-Tuning Penuh | Paling lama | Tinggi | Lebih tinggi | Domain sangat berbeda, data cukup |

**Pertanyaan Refleksi**
- Jika analogi berjalan → jogging → sprint kamu terapkan, kapan kamu merasa siap naik tingkat dan mengapa?
- Bagaimana mempersiapkan data tambahan (mis. augmentasi) agar transisi “lari” lebih stabil?
- Jika performa tidak meningkat setelah fine-tuning, langkah diagnostik apa yang akan kamu coba terlebih dahulu?


## K. Checklist Pemahaman

Gunakan daftar berikut untuk mengecek pemahamanmu sebelum lanjut ke materi berikutnya.

- [ ] Kamu dapat menjelaskan definisi dan manfaat transfer learning dengan bahasamu sendiri.
- [ ] Kamu bisa membedakan freeze vs fine-tune beserta implikasi jumlah parameter yang dilatih.
- [ ] Kamu tahu cara mengubah konfigurasi `freeze_until` serta menjelaskan dampaknya.
- [ ] Kamu menemukan lokasi file output (`outputs/figures` dan `outputs/reports`) dan memahami isinya.
- [ ] Kamu mampu menyebutkan minimal satu ide penerapan transfer learning pada bidang studi masing-masing.


## L. Referensi

- Slide: **Deep Learning 06 — Transfer Learning (Tatap Muka)** — ringkasan definisi, manfaat, jenis-jenis, serta strategi fine-tuning vs freeze yang digunakan dalam praktikum ini.
- Buku: *Deep Learning* (Goodfellow et al.) Bab 15 mengenai representasi dan transfer.
- Artikel: Pan dan Yang (2010) *A Survey on Transfer Learning* untuk memberikan landasan teori lebih formal.
- Dokumentasi PyTorch Transfer Learning Tutorial — contoh resmi yang dapat dijadikan bacaan tambahan mahasiswa.


## M. Panduan Langkah Transfer Learning

Gunakan tahapan berikut sebagai rambu sederhana saat menerapkan transfer learning dari ImageNet ke Pascal VOC2007. Fokuslah pada konsep; kita menjaga beban komputasi tetap ringan.

1. **Pilih backbone pretrained ImageNet.** Contoh: `torchvision.models.resnet18(weights="IMAGENET1K_V1")`. Model ini sudah memahami bentuk umum seperti tepi, tekstur, dan pola objek.
2. **Siapkan dataset target ringan.** VOC2007 dapat dimuat dengan subset kecil atau hanya beberapa kelas. Gunakan transformasi yang menyamakan ukuran dan normalisasi ke standar ImageNet.
3. **Ganti classifier head.** Sesuaikan jumlah output dengan label VOC2007 (misal 20 kelas). Pada tahap ini kamu ibarat pemain biola yang mengganti skor musik menjadi pseudocode sementara ritme tetap sama.
4. **Beku-kan layer awal.** Mulailah dengan mode feature extraction: matikan grad pada sebagian besar backbone sehingga hanya head yang belajar. Ini seperti berjalan santai sambil memegang biola, memindahkan rasa ritme ke latihan mengetik.
5. **Latih dengan epoch singkat.** Gunakan batch kecil dan 3–5 epoch terlebih dahulu. Observasi loss dan akurasi; catat apa yang berubah.
6. **Buka beberapa layer akhir bila diperlukan.** Jika performa stagnan, buka blok terakhir dan turunkan learning rate. Kini kamu beralih dari berjalan ke jogging, memadukan intuisi musikal dengan logika algoritmik.
7. **Evaluasi dan catat insight.** Fokus pada apa yang kamu pelajari mengenai representasi fitur, bukan sekadar angka. Tanyakan: bagian mana yang paling banyak mentransfer pengetahuan?

Iterasikan langkah-langkah ini secara bertahap; setiap siklus bagaikan mencoba genre musik baru sambil tetap membawa dasar teknik yang sama.


In [26]:
%%bash
cat <<'EOF'
Analogi Transfer Learning:
- Dari main biola ke ngoding: teknik mengenali pola irama dialihkan jadi logika if-else dan loop.
- Dari berjalan santai ke sprint: mulai dengan freeze yang aman, lanjutkan fine-tune saat siap berlari.
- Dari belajar bahasa serumpun ke bahasa baru: gunakan kosakata lama sebagai batu loncatan.
EOF


Analogi Transfer Learning:
- Dari main biola ke ngoding: teknik mengenali pola irama dialihkan jadi logika if-else dan loop.
- Dari berjalan santai ke sprint: mulai dengan freeze yang aman, lanjutkan fine-tune saat siap berlari.
- Dari belajar bahasa serumpun ke bahasa baru: gunakan kosakata lama sebagai batu loncatan.


## N. Glosarium & Istilah Kunci

- **Transfer Learning:** Teknik memanfaatkan model terlatih pada tugas berbeda sebagai titik awal tugas baru.
- **Backbone:** Bagian utama jaringan (biasanya konvolusional) yang mengekstraksi fitur dari input.
- **Classifier Head:** Lapisan akhir yang mengubah fitur menjadi prediksi kelas.
- **Fine-Tuning:** Proses melatih ulang (sebagian) bobot pretrained agar menyesuaikan domain target.
- **Freeze:** Menonaktifkan update grad pada layer tertentu.
- **Feature Extraction:** Strategi menggunakan backbone beku untuk mengambil fitur lalu melatih classifier baru.
- **Domain Gap:** Perbedaan karakteristik antara data sumber dan target; makin lebar gap, makin banyak adaptasi diperlukan.
- **Early Stopping:** Teknik menghentikan training saat validasi tidak membaik untuk mencegah overfitting.
